In [1]:
import pathlib
import sys 
import random
import json
import cv2 
import base64
import requests as req
import os
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import cm
from matplotlib.colors import ListedColormap, LinearSegmentedColormap
from io import BytesIO  
%matplotlib inline
#To compute intersection of lines
import shapely
from shapely.geometry import LineString, Point
from scipy.spatial import distance

In [21]:
#constants
PART_NAMES = [
    "nose", "leftEye", "rightEye", "leftEar", "rightEar", "leftShoulder",
    "rightShoulder", "leftElbow", "rightElbow", "leftWrist", "rightWrist",
    "leftHip", "rightHip", "leftKnee", "rightKnee", "leftAnkle", "rightAnkle"
]
PART_IDS = {pn: pid for pid, pn in enumerate(PART_NAMES)} ;PART_IDS
CONNECTED_PART_NAMES = [
    ("leftHip", "leftShoulder"), ("leftElbow", "leftShoulder"),
    ("leftElbow", "leftWrist"), ("leftHip", "leftKnee"),
    ("leftKnee", "leftAnkle"), ("rightHip", "rightShoulder"),
    ("rightElbow", "rightShoulder"), ("rightElbow", "rightWrist"),
    ("rightHip", "rightKnee"), ("rightKnee", "rightAnkle"),
    ("leftShoulder", "rightShoulder"), ("leftHip", "rightHip")
]

CONNECTED_PART_INDICES = [(PART_IDS[a], PART_IDS[b]) for a, b in CONNECTED_PART_NAMES];

CONNECTED_EYELINE_NAMES = [
    ("rightShoulder", 'leftEye') , ('rightEar', 'rightEye')
]
CONNECTED_EYELINE_INDICES = [(PART_IDS[a], PART_IDS[b]) for a, b in CONNECTED_EYELINE_NAMES];

VISION_LINE = []

In [20]:
len(PART_NAMES)

17

#Helper functions

In [3]:
def getBase64Image(filePath):
    with open(filePath, "rb") as img_file:
        my_string = base64.b64encode(img_file.read())
        my_string = my_string.decode('utf-8')
    return my_string

In [4]:
def convertToBase64(image):
    retval, buffer = cv2.imencode('.jpg', image)
    my_string = base64.b64encode(buffer)
    my_string = my_string.decode('utf-8')
    return my_string

In [5]:
def get_adjacent_keypoints(keypoints):
    '''
    Helper function of draw_skel_and _kp
    Returns 2 coord of 2 points where line needs to be drawn
    EXAMPLE: [[X1,Y1],[X2,Y2]]
    '''
    results = []
    for left, right in CONNECTED_PART_INDICES:
       results.append(
           np.array([
                [ keypoints[left]['position']['x'] , keypoints[left]['position']['y'] ],
                [ keypoints[right]['position']['x'] , keypoints[right]['position']['y'] ]
                    ]
           ).astype(np.int32)
        )
    return results
            

In [6]:
def get_adjacent_keypoint_eyeline(keypoints):
    results = []
    for left, right in CONNECTED_EYELINE_INDICES:
       results.append(
           np.array([
                [ keypoints[left]['position']['x'] , keypoints[left]['position']['y'] ],
                [ keypoints[right]['position']['x'] , keypoints[right]['position']['y'] ]
                    ]
           ).astype(np.int32)
        )
    return results

In [7]:
#Sending data to server
def getPoses(image_string):
    url2 = 'http://localhost:3000/postImage'
    data = {'imgBase64':'data:image/png;base64,'+image_string}
    r = req.post(url=url2 , data = data)
    poses = r.json()
    return poses

In [8]:
def draw_keypoints(poses,img):
    cv_keypoints= []
    for pose in poses['detectionList']:
        keypoints = pose['keypoints']
        for keypoint in keypoints:
            x,y,score = round(keypoint['position']['x']) ,round(keypoint['position']['y']),keypoint['score']
            cv_keypoints.append(cv2.KeyPoint(x,y , 10. *score))
    out_img = cv2.drawKeypoints(img, cv_keypoints , outImage=np.array([]) ) 

In [9]:
def draw_skelton(poses,img):
    '''
    This function is not used anywhere. 
    Written to demonstrate how skelton can be drawn
    '''
    adjacent_keypoints = []
    out_img = img
    for pose in poses['detectionList']:
        keypoints  = pose['keypoints']
        new_keypoints = get_adjacent_keypoints(keypoints)
        adjacent_keypoints.extend(new_keypoints)
    out_img = cv2.polylines(out_img , adjacent_keypoints, isClosed=False, color=(255,255,0))
    return out_img    

In [10]:
def draw_skel_and_kp(poses , img, color):
    out_img = img
    adjacent_keypoints = []
    cv_keypoints = []
    #For every pose of the player
    for pose in poses['detectionList']:
        keypoints = pose['keypoints']
        new_keypoint = get_adjacent_keypoints(keypoints)
        adjacent_keypoints.extend(new_keypoint)
        
        for keypoint in keypoints:
            x,y,score = round(keypoint['position']['x']) ,round(keypoint['position']['y']),keypoint['score']
            cv_keypoints.append(cv2.KeyPoint(x,y , 10. * score))
    out_img = cv2.drawKeypoints(
        img, cv_keypoints , outImage=np.array([]), color=color,
        flags=cv2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS )
    out_img = cv2.polylines(out_img , adjacent_keypoints , isClosed=False, color=list(color))
    return out_img

In [11]:
def draw_extended_line(out_img,p1,p2,color):
    '''
    Helper function of draw_visibility_cone
    Given 2 points and 1 image, draw a lines extends it
    '''
    theta1 = np.arctan2(p1[1]-p2[1], p1[0]-p2[0]) 
    endpt_x = int(p1[0] - 1000*np.cos(theta1))
    endpt_y = int(p1[1] - 1000*np.sin(theta1))
    cv2.line(out_img, (p1[0], p1[1]), (endpt_x, endpt_y), color, 2)
    
    return out_img,theta1,[endpt_x, endpt_y]
    

In [24]:
def get_line_object(p1,p2):
    '''
    Helper function of get_line_intersection
    '''
    return LineString([[p1[0] , p1[1]], [p2[0] , p2[1]] ])
    

def get_line_intersection(line1,line2):
    '''
    Helper function of drawvisibilty_cone
    This function finds  2 line intersects or not.
    Parameter 
    ---------
    line1 : A line object
    line2 :  A line object
    
    Return
    ------------
    p5 : The list containing intersection point [x,y]
    
    False  : There is no intersection 
   
    '''
    try:
        int_pt = line1.intersection(line2)
        #print(int_pt)
        p5 = [int_pt.x, int_pt.y]        
    except AttributeError:
        #print("Returing None")
        return False
    return p5

In [13]:
def find_angle_between_2_line(p1,p2,p3,p4):
    v1 = np.array(p1) - np.array(p2)
    v2 = np.array(p3) - np.array(p4)
    
    cosine_angle = np.dot(v1, v2) / (np.linalg.norm(v1) * np.linalg.norm(v2))
    angle = np.arccos(cosine_angle)
    return angle

In [20]:
def draw_general_line(out_img,p5,theta3,color):
    '''
    Helper functino of draw_visibilty_cone
    
    '''
    endpt_x = int(p5[0] - 1000*np.cos(theta3))
    endpt_y = int(p5[1] - 1000*np.sin(theta3))
    cv2.line(out_img, (int(p5[0]), int(p5[1])), (endpt_x, endpt_y), color, 2)
    return out_img
def get_general_line(p5,theta3):
    '''
    This function find the the middle/general line
    Parameter
    ---------
    p5 : The intersection point of the player cone lines 
    theta3 : The angle that the new line need to be in
    
    Return 
    -----------
    line1 : A line object. The middle line
    '''
    endpt_x = int(p5[0] - 1000*np.cos(theta3))
    endpt_y = int(p5[1] - 1000*np.sin(theta3))
    line1 = get_line_object([[p5[0] , p5[1]], [endpt_x , endpt_y]])
    return line1

In [15]:
def get_positive_theta(theta):
    if 0<=theta<=np.pi:
        return theta
    else:
        if -np.pi/2 <=theta < 0:
            theta = 2*np.pi - abs(theta)
        elif -np.pi <= theta < np.pi/2:
            theta = (3/4*np.pi) - abs(theta)
    return theta
            
        

In [25]:
def get_cone_line_intersection(keypoints):
    '''
    Helper function of set_vision_lines()
    Parameter
    ---------
    keypoints : The keypoint dictionary containing all the keypoints 
    
    Return 
    ---------
     p5 : The intersection point 
    
    False  : There is no intersection 
    '''
    new_keypoints = get_adjacent_keypoint_eyeline(keypoints)
    p1,p2 = new_keypoints[0][0],new_keypoints[0][1]
    p3,p4 = new_keypoints[1][0],new_keypoints[1][1]
    #Get the cone line intersection
    line1 = get_line_object(p1,p2)
    line2 = get_line_object(p3,p4)
    p5 = get_line_intersection(line1,line2)
    return p5


def set_vision_lines(posenetPred):
    '''
    This function will update the GLOBAL VARIABLE VISION_LINE
    It will remove the vision line for the last frame
    It will add vision line for this frame for all the players
    
    Parameter
    ----------
    posenetPred : The output from poenet. This should contain keypoints that are relative to
                  the big image. 
                  
    
    '''
    global VISION_LINE
    VISION_LINE = []
    for pose in poses['detectionList']:
        keypoints = pose['keypoints']
        #Get cone line intersection point
        p5 = get_cone_line_intersection(keypoints)
       # get mid line 
        if p5 != False:
            theta3 = (theta1 + theta2)/2
            general_line = get_general_line(p5, theta3)
            VISION_LINE.append(general_line)
        else:
            VISION_LINE.append(False)
    print(f"Successfully initalized general line:{len(VISION_LINE)}")



In [29]:
def get_bbox_coords(bbox):
    '''
    This function will return all the 4 coords of the bouding boxes
    Parameter
    ---------
    bbox : [left, top ,right, bottom]
    
    Return
    --------
    list : [left_top , right_bottom , left_bottom , right_top]
    '''
    left_top,right_bottom = (bbox[0] , bbox[1]), (bbox[2] , bbox[3])
    left_bottom , right_top = (left_top[0] ,right_bottom[1]) , (right_bottom[0] , left_top[1])
    return [left_top , right_bottom , left_bottom , right_top]
    
    
    
def get_bbox_diagonals(left_top , right_bottom , left_bottom , right_top):
    '''
    This fuction finds the 2 diagonal of the bounding boxes
    Parameter
    ----------
    left_top , right_bottom , left_bottom , right_top : The 4 points of the bbox
    
    Return
    --------
    diagonal1 : A line object. It starts from left_top , right_bottom 
    diagonal2 : A line object. It starts from left_bottom , right_top
    '''
    diagonal1 = get_line_object(left_top , right_bottom)
    diagonal2 = get_line_object(left_bottom , right_top)
    return diagonal1 , diagonal2


def check_bbox_intersection(bbox , lines):
    '''
    Helper function of check_bboxs_intersection
    This function find out the number of vision line that intersect with bbox
    
    Parameter
    -----------
    bbox : A list [left,top,right,botoom] coord in order
    lines : VISION_LINE, where the vision line of the player in the bbox is set to False. 
            Because we dont want to consider it's intersection with it self
    '''
    count = 0
    bbox_points = get_bbox_coords(bbox)
    diagonal1, diagonal2 = get_bbox_diagonals(*bbox_points)
    for line in lines:
        if line != False: 
            intersection1 = get_line_intersection(diagonal1,line)
            intersection2 = get_line_intersection(diagonal2,line)
            if intersection1 or intersection2:
                count += 1
    return count


def check_bboxs_intersection(bboxs):
    '''
    This function finds the number of VISION_LINES each bboxs is intersected by 
    Parameter
    ----------
    bboxs : A list with bbox. 
            Example: [[left,top,right,botoom],.......[left,top,right,botoom],.. ] 
            
    Return
    ---------
    counter : A list with the number of VISION_LINES each bboxs is intersected by
              Example: [8,....]
    '''
    global VISION_LINE
    counter = []
    for index,bbox in enumerate(bboxs):
        VISION_LINE_COPY = VISION_LINE[:]
        VISION_LINE_COPY[index] = False
        count = check_bbox_intersection(bbox, VISION_LINE_COPY)
        counter.append(count)
    return counter


        
            
    
    

In [16]:
def draw_visibilty_cone(poses,img,color=None):
    out_img = img
    adjacent_keypoints = []
    cv_keypoints = []
    counter = 0
    for pose in poses['detectionList']:
        keypoints = pose['keypoints']
        new_keypoints = get_adjacent_keypoint_eyeline(keypoints)
        p1,p2 = new_keypoints[0][0],new_keypoints[0][1]
        p3,p4 = new_keypoints[1][0],new_keypoints[1][1]
       # Draw the two lines
        out_img,theta1,p2 = draw_extended_line(out_img,p1,p2,[255,255,0])
        out_img,theta2,p4= draw_extended_line(out_img,p3,p4,[0,255,0])
       # get mid line 
#         p5 = get_line_intersection(p1,p2,p3,p4)
#         if p5 != None:
#             theta3 = (theta1 + theta2)/2
#             if counter <100:
#                 draw_general_line(out_img,p5,theta3,[0,0,255])
#             counter += 1
    return out_img

In [17]:
p5 = [536.3247184387701, 328.8307943698096]
draw_general_line(out_img,p5,np.radians(120.06858282186246),[255,0,255])
cv2.imwrite('/Users/sandeep/Desktop/dataandmodles/data/cavsCone.JPG' ,out_img) 

NameError: name 'out_img' is not defined

#Sending to server

In [17]:
#Loading model
r = req.get(url='http://localhost:3000/loadModel')

In [18]:

img = cv2.imread('/Users/sandeep/Desktop/dataandmodles/data/cropped.png')
myString =convertToBase64(img)
poses = getPoses(myString)
out_img = draw_skel_and_kp(poses,img,[255,0,0])
#out_img = draw_visibilty_cone(poses,img)
cv2.imwrite('/Users/sandeep/Desktop/dataandmodles/data/croppedPose.JPG' ,out_img) 

True

In [18]:
videoPath ='/Users/sandeep/Desktop/dataandmodles/data/3-Pointer2.mov'
cap = cv2.VideoCapture(videoPath)
count = 0
while cap.isOpened():
    ret,frame = cap.read()
    if ret:
        #do stuff
        myString =convertToBase64(frame)
        poses = getPoses(myString)
        frame = draw_visibilty_cone(poses,frame)
        frame = draw_skel_and_kp(poses,frame,[255,0,0])
#         End of do stuff
        cv2.imshow('test', frame)
        #cv2.imwrite(f"/Users/sandeep/Desktop/dataandmodles/data/coneVision/{count}.png" , frame)
        count += 1
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break  
    else:
        cap.release()
        break
cap.release()
cv2.destroyAllWindows()
